This notebook is intended to demonstrate how vessel segmentation methods of ITKTubeTK can be applied to multi-channel MRI (MRA + T1, T2, etc).

In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [2]:
ImageType = itk.Image[itk.F, 3]

im1 = itk.imread("Segmentation.mha")
resample = ttk.ResampleImage.New(Input=im1, MakeIsotropic=True)
resample.Update()
im1Iso = resample.GetOutput()

In [3]:
imMath = ttk.ImageMath.New(im1Iso)
imMath.Threshold(1,1,0,1)
im1Inv = imMath.GetOutputFloat()
distFilter = itk.DanielssonDistanceMapImageFilter.New(im1Inv)
distFilter.SetUseImageSpacing(True)
distFilter.SetInputIsBinary(True)
distFilter.SetSquaredDistance(False)
distFilter.Update()
im1Dist = distFilter.GetOutput()

view(im1Dist)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [10]:
numSeeds = 200

vSeg = ttk.SegmentTubes.New(Input=im1Dist)
#vSeg.SetVerbose(True)
vSeg.SetMinCurvature(0.0)
vSeg.SetMinRoundness(0.0)
vSeg.SetMinRidgeness(0.9)
vSeg.SetMinLevelness(0.0)
vSeg.SetRadiusInObjectSpace( 0.4 )
vSeg.SetSeedMask( im1Dist )
vSeg.SetSeedRadiusMask( im1Dist )
vSeg.SetOptimizeRadius(False)
vSeg.SetUseSeedMaskAsProbabilities(True)
vSeg.SetSeedExtractionMinimumProbability(0.4)
#vSeg.SetSeedMaskMaximumNumberOfPoints( numSeeds )
vSeg.ProcessSeeds()

In [11]:
tubeMaskImage = vSeg.GetTubeMaskImage()
view(tubeMaskImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [12]:
SOWriter = itk.SpatialObjectWriter[3].New(vSeg.GetTubeGroup())
SOWriter.SetFileName( "BinaryImageVessels.tre" )
SOWriter.Update()